In [8]:
from langchain_ollama import OllamaLLM
from langchain.prompts import ChatPromptTemplate

llm = OllamaLLM(model="llama3.2:3b")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    ("user", "{question}")
])
chain = prompt | llm 

In [9]:
print(chain.invoke({"question": "Explain Retrieval-Augmented Generation"}))

Retrieval-Augmented Generation (RAG) is a novel approach to natural language processing (NLP) that combines the strengths of retrieval-based models and generation-based models. The goal of RAG is to generate high-quality text by leveraging pre-existing knowledge in a large corpus, often referred to as the "knowledge base" or "Knowledge Graph".

Here's how it works:

1. **Retrieval**: The RAG model starts by performing a retrieval operation on the knowledge base to find relevant documents that match the input prompt. This is done using a query-based approach, where the input prompt is used as a search query to retrieve relevant documents.
2. **Ranking**: The retrieved documents are then ranked based on their relevance to the input prompt. The top-ranked document(s) are selected for further processing.
3. **Generation**: The RAG model generates text by combining the features of the top-ranked document with new, generated content. This is done using a generation-based approach, where the 

In [11]:
import requests 
from langchain.tools import Tool 


def call_mcp_add(query:str):
    r = requests.post("http://localhost:8000/add_data",json={"query":query})
    return r.json()

def call_mcp_read(query:str):
    r = requests.post("http://localhost:8000/read_data",json={"query":query})
    return r.json()


mcp_add_tool = Tool.from_function(
    func=call_mcp_add,
    name="add_data",
    description="Add a new row to SQLite DB using MCP tool."
)

mcp_read_tool = Tool.from_function(
    func=call_mcp_read,
    name="read_data",
    description="Read rows from SQLite DB using MCP tool."
)


In [12]:
from langchain.agents import initialize_agent

tools = [mcp_add_tool,mcp_read_tool]
agent = initialize_agent(
    tools,
    llm,
    agent = "zero-shot-react-description",
    verbose = True 
)

/tmp/ipykernel_3887/2249516843.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [17]:
print(agent.run("Add to a database: Nguyen Nguyen whose age is 22 and is a tennis player"))




> Entering new AgentExecutor chain...


Question: Add to a database: Nguyen Nguyen whose age is 22 and is a tennis player
Thought: To add this person's information to the database, I need to create a new row with relevant fields.
Action: add_data
Action Input: query="INSERT INTO people (name, age, sport) VALUES ('Nguyen Nguyen', 22, 'tennis')"
Observation: {'detail': [{'type': 'missing', 'loc': ['query', 'query'], 'msg': 'Field required', 'input': None}]}
Thought:Question: Add to a database: Nguyen Nguyen whose age is 22 and is a tennis player
Thought: Question: Add to a database: Nguyen Nguyen whose age is 22 and is a tennis player
Thought: To add this person's information to the database, I need to create a new row with relevant fields.
Action: add_data
Action Input: query="INSERT INTO people (name, age, sport) VALUES ('Nguyen Nguyen', 22, 'tennis')"
Observation: {'detail': [{'type': 'missing', 'loc': ['query', 'query'], 'msg': 'Field required', 'input': None}]}
Thought:Question: Add to a database: Nguyen Nguyen whose age 